In [305]:
import hashlib
from random import randint
from sage.interfaces.ecm import *

m1 = "Grigori loh"
m2 = "Grigory ne loh"
Hm1 = int((hashlib.md5(m1.encode("utf-8")).hexdigest()), 16)
Hm2 = int((hashlib.md5(m2.encode("utf-8")).hexdigest()), 16)
print(Hm1, Hm2)

p = random_prime(1000000, lbound = 900000 )
print()
print(p)

def PK_and_SK(p):
    """
    Создание открытого ключа
    
    """
    q = ecm.factor((p-1) / 2 )[-1]
    x = randint(2, q - 1)
    g = 1
    while g == 1:
        h = randint(1, p - 2)
        g = pow(h,int((p -1) / q), p)
    y = pow(g, x, p)
    return x, q, g,  y
print(PK_and_SK(p))

def sign(p, q, x, g , mes, k):
    if k > q:
        return 'k > q'
    r = int(int(pow(g, k, p)) % q)
    k_minus1 = inverse_mod(k, q)
    Hm = int((hashlib.md5(mes.encode("utf-8")).hexdigest()), 16)
    s = int((k_minus1 * (Hm + x * r))) % q
    return Hm, r, s

def sign_check(r, s, q, Hm, y, x):
    w = inverse_mod(s, q)
    u1 = int((Hm * w)) % q
    u2 = int((r * w)) % q
    v  = int(pow(g, u1 + u2 * x, p)) % q
    v = int(pow(g, u1, p)*pow(y, u2, p)) % q 
    return v, r


def secret(Hm1, Hm2, s1, s2, q):
    """
    Поиск серкретного x по двум мообщениям подписанным одним (r, k)
        
    """
    
    x = (Hm1 * inverse_mod(s1, q) - Hm2 * inverse_mod(s2, q)) *\
    inverse_mod(r * (inverse_mod(s2, q) - inverse_mod(s1, q)), q) % q
    
    return x

45477745298662712314192239555914263435 164928248859189475376152111395925683749

943781
(15254, 47189, 113679, 353673)


In [306]:
m1 = 'Grigori loh'
m2 = 'Grigori ne loh'
x, q, g,  y = PK_and_SK(p)
print(x, q, g, y)
k = 13
Hm1, r1, s1 = sign(p, q, x, g, m1, k)
Hm2, r2, s2 = sign(p, q, x, g, m2, k)
print(Hm1, r1, s1)
print(Hm2, r2, s2)

v1, r3 = sign_check(r1, s1, q, Hm1, y, x)
v2, r4 = sign_check(r2, s2, q, Hm2, y, x)

print(v1, r3)
print(v2, r4)

x_isk = secret(Hm1, Hm2, s1, s2, q)
print(x, x_isk)

3309 47189 619733 425923
45477745298662712314192239555914263435 28936 43921
201291268780051351498871390518114340427 28936 283
28936 28936
28936 28936


TypeError: argument of type 'sage.rings.integer.Integer' is not iterable

In [243]:
w1 = inverse_mod(s1, q)
w2 = inverse_mod(s2, q)
u11 = int((Hm1 * w1)) % q
u12 = int((Hm2 * w2)) % q
u21 = int((r * w1)) % q
u22 = int((r * w2)) % q
print(int(pow(g, (u11 + x * u21), p)) % q) 
print(int(pow(g, (u12 + x * u22), p)) % q)

93
263


In [229]:
w1 = inverse_mod(s1, q)
u11 = int((Hm1 * w1)) % q
u21 = int((r * w1)) % q
v1  = int(pow(g, u11 + u21 * x, p)) % q

w2 = inverse_mod(s2, q)
u12 = int((Hm2 * w2)) % q
u22 = int((r * w2)) % q
v2  = int(pow(g, u12 + u22 * x, p)) % q

print(v, v1)

1635 59552


In [265]:
def sign_steal(r, s, q, Hm, y, x):
    w = inverse_mod(s, q)
    u1 = int((Hm * w)) % q
    u2 = int((r * w)) % q
    v  = int(pow(g, u1 + u2 * x, p)) % q
    return v, r, u1, u2, w

v, r, u1, u2, w1 = sign_steal(r, s1, q, Hm1, y, x)
v, r, u12, u22, w2 = sign_steal(r, s2, q, Hm2, y, x)

print(v, r, u12, u22, w2)

deg1 = (u11 + x * u21)
deg2 = (u12 + x * u22)

print(int(pow(g, deg1, p)) %q,  int(pow(g, deg2, p)) % q)
v1 = int(pow(g, u11, p)*pow(y, u21, p)) % q 
v2 = int(pow(g, u11, p)*pow(y, u22, p)) % q 
print()
v11  = int(pow(g, u11 + u21 * x, p)) % q
v22  = int(pow(g, u12 + u22 * x, p)) % q
print(v1, v2)
print(v11, v22)

34981 411386 4255 49015 7302
34981 411386 42811 31797 42282
29812 34981

29812 22181
29812 34981


In [270]:
def sign_steal(r, s, q, Hm, y, x):
    w = inverse_mod(s, q)
    u1 = int((Hm * w)) % q
    u2 = int((r * w)) % q
    v  = int(pow(g, u1 + u2 * x, p)) % q
    return v, r, u1, u2, w

v, r, u1, u2, w = sign_steal(r, s1, q, Hm1, y, x)
print(v, r, u1, u2, w)
v, r, u1, u2, w = sign_steal(r, s2, q, Hm2, y, x)
print(v, r, u1, u2, w)

59036 411386 30839 51078 11698
11189 411386 28399 43760 59927
